In [3]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from scipy import stats
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from collections import Counter
import scanpy as sc
import anndata as an
import sys
sys.path.append('/mnt/c/MyPrograms/__github/scycle/')
import scycle as cc

import seaborn as sns

def smooth_adata_by_pooling(adata,X_embed,n_neighbours=10):
    adata_pooled = adata.copy()
    nbrs = NearestNeighbors(n_neighbors=n_neighbours).fit(X_embed)
    distances, indices = nbrs.kneighbors(X_embed)    
    adata_pooled.X = smooth_matrix_by_pooling(get_nd_array(adata.X),indices)
    if 'matrix' in adata.layers:
        adata_pooled.layers['matrix'] = smooth_matrix_by_pooling(get_nd_array(adata.layers['matrix']),indices)
    if 'spliced' in adata.layers:
        adata_pooled.layers['spliced'] = smooth_matrix_by_pooling(get_nd_array(adata.layers['spliced']),indices)
    if 'unspliced' in adata.layers:
        adata_pooled.layers['unspliced'] = smooth_matrix_by_pooling(get_nd_array(adata.layers['unspliced']),indices)
    return adata_pooled

def smooth_matrix_by_pooling(matrix,indices):
    matrix_pooled = matrix.copy()
    for i in range(len(indices)):
        matrix_pooled[i,:] = np.mean(matrix[indices[i],:],axis=0)
    return matrix_pooled

def get_nd_array(arr):
    x = None
    if str(type(arr)):
        x = arr
    else:
        x = arr.toarray()
    return x

    
def preprocessing_without_pooling(adata):
    if Normalize_Totals:
        sc.pp.normalize_total(adata, target_sum=10000)
    if top_variable_genes>0:
        #sc.pp.highly_variable_genes(adata,n_top_genes=top_variable_genes,n_bins=20)
        #ind_genes = np.where(adata.var['highly_variable'])[0]
        vars = np.var(adata.X,axis=0)
        inds = np.flip(np.argsort(vars))
        ind_genes = inds[0:top_variable_genes]
        if 0 in vars[ind_genes]:
            ind_first_zero = np.argwhere(vars[ind_genes]==0)[0][0]
            ind_genes = ind_genes[0:ind_first_zero]
        #print(vars[ind_genes])
        adata = adata[:,ind_genes]
    if not Already_Log_Transformed:
        sc.pp.log1p(adata)
    sc.tl.pca(adata,n_comps=number_of_pcs)
    return adata

# pooling procedure
def pooling_procedure(adata):
    if n_neighbours_for_pooling>0:    
        adata_work = adata_orig.copy()
        preprocessing_without_pooling(adata)
        sc.tl.pca(adata,n_comps=number_of_pcs)
        X_pca = adata.obsm['X_pca']
        adata = smooth_adata_by_pooling(adata_work,X_pca,n_neighbours=n_neighbours_for_pooling)
    return adata

def preprocessing_dataset(adata):
    adata = preprocessing_without_pooling(adata)    
    sc.tl.pca(adata,n_comps=number_of_pcs)
    display(adata)
    return adata

def ismember(A, B):
    dct = {}
    for s,i in enumerate(B):
        dct[i] = s
    return [ dct[a] for a in A ]

def load_signature_file(file):
    sigs = {}
    with open(file,'r',encoding="utf8",errors='ignore') as fin:
        line = fin.readline().strip('\n').strip(' ')
        while line:
            parts = line.split('\t')
            lst = parts[2:]
            lst = [s.split('[')[0] for s in lst if not s=='']
            sigs[parts[0]] = lst
            line = fin.readline().strip('\n').strip(' ')
    return sigs

def load_weighted_signature_file(file):
    sigs = {}
    with open(file,'r',encoding="utf8",errors='ignore') as fin:
        line = fin.readline().strip('\n').strip(' ')
        while line:
            parts = line.split('\t')
            lst = parts[2:]
            lst1 = [s.split('[')[0] for s in lst if not s=='']
            weights = [float(s.split('[')[1].split(']')[0]) for s in lst if not s=='']
            #print(lst1,weights)
            sigs[parts[0]] = (lst1,weights)
            #sigs[parts[0]+'_W'] = weights
            line = fin.readline().strip('\n').strip(' ')
    return sigs

def calc_scores(anndata,signature_dict):
    matrix = anndata.to_df().to_numpy()
    scores_dic = {}
    for key in signature_dict:
        names = np.array(signature_dict[key])
        inds = np.where(np.isin(anndata.var_names,names))[0]
        matrix_sel = matrix[:,inds]
        scores = np.mean(matrix_sel,axis=1)
        scores_dic[key] = scores
    return scores_dic

def calc_weighted_scores(anndata,signature_dict):
    matrix = anndata.to_df().to_numpy()
    scores_dic = {}
    for key in signature_dict:
        names_weights = signature_dict[key]
        names = names_weights[0]
        weights = np.array(names_weights[1])
        inds = np.where(np.isin(anndata.var_names,names))[0]
        sel_names = anndata.var_names[inds]
        ind_in_names = ismember(sel_names,names)
        names1 = np.array(names)[ind_in_names]
        weights1 = np.array(weights)[ind_in_names]
        inds = ismember(names1,anndata.var_names)
        matrix_sel = matrix[:,inds]
        gene_means = np.mean(matrix_sel,axis=0)
        meanmat = np.outer(np.ones(matrix_sel.shape[0]),gene_means)
        matrix_sel = matrix_sel-meanmat
        scores = np.matmul(matrix_sel,weights1)
        scores_dic[key] = scores
    return scores_dic


def calc_histone_score(adata2k):
    histone_names1 = np.argwhere(adata2k.var_names.str.startswith('H1'))
    histone_names2 = np.argwhere(adata2k.var_names.str.startswith('H2'))
    histone_names3 = np.argwhere(adata2k.var_names.str.startswith('H3'))
    histone_names4 = np.argwhere(adata2k.var_names.str.startswith('H4'))
    histone_names5 = np.argwhere(adata2k.var_names.str.startswith('HIST'))
    histone_names = np.union1d(np.union1d(histone_names1,histone_names2),np.union1d(histone_names3,histone_names4))
    histone_names = np.union1d(histone_names,histone_names5)
    histone_names = adata2k.var_names[histone_names]
    print('Found histone genes:',*histone_names)
    inds_histones = np.where(np.isin(adata2k.var_names,histone_names))[0]
    matrix = adata2k.to_df().to_numpy()
    matrix_sel = matrix[:,inds_histones]
    scores = np.mean(matrix_sel,axis=1)
    return scores

def _compute_ica(adata,thr=2.0):
    adata.uns["scycle"] = {}
    cc.tl.dimensionality_reduction(adata,method='ica')

    idx_g1s = adata.uns['scycle']['find_cc_components']['indices']['G1-S']
    adata.uns['S-phase_genes'] = list(adata.var_names[adata.uns['dimRed'].S_[idx_g1s,:]>3])
    idx_g2m = adata.uns['scycle']['find_cc_components']['indices']['G2-M']
    adata.uns['G2-M_genes'] = list(adata.var_names[adata.uns['dimRed'].S_[idx_g2m,:]>3])
    #idx_g2m_inh = adata.uns['scycle']['find_cc_components']['indices']['G2-M-']
    #adata.uns['G2-M_INH_genes'] = list(adata.var_names[adata.uns['dimRed'].S_[idx_g2m_inh,:]>3])
    #idx_histone = adata.uns['scycle']['find_cc_components']['indices']['Histone']
    #adata.uns['Histone_IC_genes'] = list(adata.var_names[adata.uns['dimRed'].S_[idx_histone,:]>3])
    signature_dict = {'S-phase':adata.uns['S-phase_genes'],
                      'G2-M':adata.uns['G2-M_genes'],
                      #'G2-M-':adata.uns['G2-M_INH_genes'],
                      #'Histone_IC':adata.uns['Histone_IC_genes']
                      }  
    sc.pp.highly_variable_genes(adata,n_top_genes=2001,n_bins=20)
    ind_genes2k = np.where(adata.var['highly_variable'])[0]
    adata2k = adata[:,ind_genes2k]
    scores_dic = calc_scores(adata2k,signature_dict)
    for score in scores_dic:
        adata.obs[score] = scores_dic[score]
    adata.varm['P_dimRed'] = adata.uns['dimRed'].S_.T
    adata.uns['dimRed'] = None

In [4]:
folder = 'data/cellline/'
cell_line = 'CVCL_0023'

In [2]:
!gsutil -m cp gs://tahoe100m_bycelllines/*{cell_line}*.h5ad data/cellline/

Copying gs://tahoe100m_bycelllines/plate10_CVCL_0023.h5ad...
Copying gs://tahoe100m_bycelllines/plate11_CVCL_0023.h5ad...                    
Copying gs://tahoe100m_bycelllines/plate12_CVCL_0023.h5ad...                    
Copying gs://tahoe100m_bycelllines/plate13_CVCL_0023.h5ad...
Copying gs://tahoe100m_bycelllines/plate14_CVCL_0023.h5ad...                    
Copying gs://tahoe100m_bycelllines/plate1_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate2_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate3_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate4_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate5_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate6_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate7_CVCL_0023.h5ad...                     
Copying gs://tahoe100m_bycelllines/plate8_CVCL_0023.h5ad...         

In [3]:
files = os.listdir(folder)
print(files)
h5s = []
for i,f in enumerate(files):
    print(i+1,f)
    adata = sc.read_h5ad(folder+f)
    h5s.append(adata)

['plate3_CVCL_0023.h5ad', 'plate2_CVCL_0023.h5ad', 'plate13_CVCL_0023.h5ad', 'plate1_CVCL_0023.h5ad', 'plate8_CVCL_0023.h5ad', 'plate11_CVCL_0023.h5ad', 'plate10_CVCL_0023.h5ad', 'plate7_CVCL_0023.h5ad', 'plate6_CVCL_0023.h5ad', 'plate4_CVCL_0023.h5ad', 'plate12_CVCL_0023.h5ad', 'plate5_CVCL_0023.h5ad', 'plate14_CVCL_0023.h5ad', 'plate9_CVCL_0023.h5ad']
plate3_CVCL_0023.h5ad
plate2_CVCL_0023.h5ad
plate13_CVCL_0023.h5ad
plate1_CVCL_0023.h5ad
plate8_CVCL_0023.h5ad
plate11_CVCL_0023.h5ad
plate10_CVCL_0023.h5ad
plate7_CVCL_0023.h5ad
plate6_CVCL_0023.h5ad
plate4_CVCL_0023.h5ad
plate12_CVCL_0023.h5ad
plate5_CVCL_0023.h5ad
plate14_CVCL_0023.h5ad
plate9_CVCL_0023.h5ad


In [4]:
adata_cellline = sc.concat(h5s)

In [5]:
adata_cellline.write_h5ad(folder+cell_line+'.h5ad',compression='gzip')

In [6]:
for h in h5s:
    del h

In [5]:
adata_cellline = sc.read_h5ad(folder+cell_line+'.h5ad')

In [ ]:
adata_cellline_sample = sc.pp.sample(adata_cellline,fraction=0.1,copy=True)
display(adata_cellline_sample)
adata_cellline_sample.X = adata_cellline_sample.X.todense()
adata_cellline_sample.write_h5ad(folder+cell_line+'_sample.h5ad',compression='gzip')

AnnData object with n_obs × n_vars = 266448 × 62710
    obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate'

/home/zinovyev/tahoe/lib/python3.11/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.


In [ ]:
top_variable_genes = 10000 # if negative then no selection of genes
Normalize_Totals = True
Already_Log_Transformed = False
n_neighbours_for_pooling = -1
number_of_pcs = 20

print('PREPROCESSING PARAMETERS:')
print('Already_Log_Transformed=',Already_Log_Transformed)
print('Normalize_Totals=',Normalize_Totals)
print('number_of_pcs=',number_of_pcs)
print('n_neighbours_for_pooling=',n_neighbours_for_pooling)
print('top_variable_genes=',top_variable_genes)

if n_neighbours_for_pooling>0:
    adatat1 = pooling_procedure(adata_cellline_sample)
else:
    adatat1 = adata_cellline_sample
adatat1 = preprocessing_dataset(adatat1)



PREPROCESSING PARAMETERS:
Already_Log_Transformed= False
Normalize_Totals= True
number_of_pcs= 20
n_neighbours_for_pooling= -1
top_variable_genes= 10000
